# Lesson 3: Handling Transactions with Pipelines

Welcome back! We're moving on to the next essential part of our Redis-based backend system project — handling transactions with pipelines. This will help us execute multiple Redis commands as a single atomic operation. Remember, you've already gotten comfortable with managing user data and leaderboards. This unit will take it a step further by optimizing these operations using pipelines.

## What You'll Build

Before we dive in, let's recap what you’ll be focusing on in this unit. The key tasks include:

- **Adding user data with expiration using pipelines**: We will group multiple commands into one pipeline to add user data more efficiently.
- **Adding scores to a leaderboard using pipelines**: Using pipelines to add scores will ensure these operations are atomically executed.
- **Executing the pipeline**: We'll ensure the grouped commands in the pipeline are executed together.

These tasks will help us understand how pipelines can enhance performance and consistency in our Redis operations.

Here's a snippet to remind you of how pipelines work:

```python
import redis
import json
from datetime import timedelta

# Connect to Redis
client = redis.Redis(host='localhost', port=6379, db=0)

# Add user data using pipeline
users = [
    {'username': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}},
    {'username': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}}
]

with client.pipeline() as pipeline:
    for user in users:
        pipeline.set(f"user:{user['username']}", json.dumps(user['data']), ex=timedelta(days=1))
    result = pipeline.execute()
    print(result)

print(client.get('user:alice'))
```

This way, all commands in the pipeline are sent to the Redis server in one batch when `pipeline.execute()` is called. This ensures that the commands are executed atomically.

Let's go! The more you practice, the better you'll get at building efficient backend systems.


## Handling User Data with Pipelines

Now that we have learned how to use transactions with pipelines, it's time to practice.

Update the given code to use Redis pipelines to add user data and scores.

```py
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data):
    client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score):
    client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]


# TODO: Modify the code bellow to use pipelines for adding user data and scores
for user in users:
    # TODO: Modify the code below to set user data into pipeline instead of calling add_user
    add_user(user['user_id'], user['data'])

    # TODO: Modify the code below to add user score into pipeline instead of calling add_score
    add_score(user['user_id'], user['score'])

    # Remember to execute the pipeline after adding all user data and scores

# Get leaderboard
print("Leaderboard:", get_leaderboard())

# Get rank and score for a specific user
for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

```

To implement Redis pipelines, we can batch multiple commands together to reduce the number of network round trips. This approach is beneficial for performance, especially when we are dealing with multiple Redis commands in succession. In this case, we can update the `add_user` and `add_score` operations for each user in a single pipeline.

Here’s the modified code:

```python
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(pipeline, user_id, data):
    pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def add_score(pipeline, user_id, score):
    pipeline.zadd('leaderboard', {user_id: score})

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Use pipeline for batch processing
with client.pipeline() as pipeline:
    for user in users:
        # Add user data to pipeline
        add_user(pipeline, user['user_id'], user['data'])
        
        # Add user score to pipeline
        add_score(pipeline, user['user_id'], user['score'])
    
    # Execute all pipelined commands
    pipeline.execute()

# Get leaderboard
print("Leaderboard:", get_leaderboard())

# Get rank and score for a specific user
for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")
```

### Explanation of Changes:

1. **Pipeline Initialization**: `with client.pipeline() as pipeline:` initiates a pipeline context manager.
2. **Passing Pipeline to Functions**: The `add_user` and `add_score` functions now accept a `pipeline` argument to batch Redis commands.
3. **Pipeline Execution**: The `pipeline.execute()` command sends all queued commands to the Redis server in one go.

This approach optimizes performance by reducing the number of network calls to the Redis server, making it suitable for cases where multiple commands need to be executed consecutively.

## Using Pipelines for Transactions

Great job so far! Let's now enhance our handling of transactions with Redis pipelines. We will start with some existing code to add user details and scores into a leaderboard.

Your task is to modify the add_user and add_score functions to accept an additional pipeline argument to ensure these operations can be added to a pipeline when necessary.

```py
import redis
import json
from datetime import timedelta


client = redis.Redis(host='localhost', port=6379, db=0)

# TODO: Modify the add_user function to accept an additional pipeline argument
def add_user(user_id, data):
    # TODO: Modify the code below to set the user data using the pipeline if it is provided
    # Otherwise, set the user data using the client
    client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

# TODO: Modify the add_score function to accept an additional pipeline argument
def add_score(user_id, score):
    # TODO: Modify the code below to add the user score using the pipeline if it is provided
    # Otherwise, add the user score using the client
    client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

with client.pipeline() as pipeline:
    for user in users:
        # TODO: Modify the line below to call add_user with the pipeline argument
        pipeline.set(f'user:{user["user_id"]}', json.dumps(user['data']), ex=timedelta(days=1))

        # TODO: Modify the line below to call add_score with the pipeline argument
        pipeline.zadd('leaderboard', {user['user_id']: user['score']})
    pipeline.execute()

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

```

To allow the `add_user` and `add_score` functions to work with or without a Redis pipeline, we can add an optional `pipeline` argument. This way, when a pipeline is provided, the function will add the command to the pipeline. Otherwise, it will use the standard `client` object directly. 

Here’s the modified code:

```python
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

def add_user(user_id, data, pipeline=None):
    # Use the provided pipeline if available; otherwise, use the client
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

def add_score(user_id, score, pipeline=None):
    # Use the provided pipeline if available; otherwise, use the client
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Using pipeline for batch processing
with client.pipeline() as pipeline:
    for user in users:
        # Call add_user with the pipeline argument
        add_user(user['user_id'], user['data'], pipeline=pipeline)
        
        # Call add_score with the pipeline argument
        add_score(user['user_id'], user['score'], pipeline=pipeline)
    
    # Execute all pipelined commands
    pipeline.execute()

# Get leaderboard
print("Leaderboard:", get_leaderboard())

# Get rank and score for a specific user
for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")
```

### Explanation of Changes:

1. **Optional `pipeline` Argument**: The `add_user` and `add_score` functions now accept an optional `pipeline` argument.
2. **Conditional Command Execution**: If `pipeline` is provided, the function uses it to execute the command. If not, it defaults to `client` to execute the command.
3. **Calling Functions with `pipeline` Argument**: In the `with client.pipeline() as pipeline:` block, we pass `pipeline=pipeline` to each function.

This setup allows flexibility in calling `add_user` and `add_score` with or without a pipeline, depending on the context.

## Handling Data with Redis Pipelines

Now that you have learned how to handle transactions with pipelines, it's time to practice.

Modify the provided code to make the pipeline argument optional for the add_user and add_score functions.

When the pipeline argument is null, use the default client setting; otherwise, use the pipeline setting.

```py
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

# TODO: Modify the add_user function to make the pipeline argument optional
def add_user(user_id, data, pipeline):
    # TODO: Implement the logic to use the pipeline argument if it's not None or use the default client setting
    pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

# TODO: Modify the add_score function to make the pipeline argument optional
def add_score(user_id, score, pipeline):
    # TODO: Implement the logic to use the pipeline argument if it's not None or use the default client setting
    pipeline.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

with client.pipeline() as pipeline:
    for user in users:
        add_user(user['user_id'], user['data'], pipeline)
        add_score(user['user_id'], user['score'], pipeline)
    pipeline.execute()

users.append({'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90})

# TODO: Add the new user 'david' to the users list with the add_user and add_score functions without using a pipeline

print("Leaderboard:", get_leaderboard())

for user in users:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")

```

Here’s the modified code with optional `pipeline` arguments for `add_user` and `add_score` functions. Now, if `pipeline` is `None`, these functions will use the default `client` to handle the command; otherwise, they’ll use the provided `pipeline`.

```python
import redis
import json
from datetime import timedelta

client = redis.Redis(host='localhost', port=6379, db=0)

# Modify add_user to make the pipeline argument optional
def add_user(user_id, data, pipeline=None):
    if pipeline:
        pipeline.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))
    else:
        client.set(f'user:{user_id}', json.dumps(data), ex=timedelta(days=1))

def get_user(user_id):
    data = client.get(f'user:{user_id}')
    return json.loads(data) if data else None

# Modify add_score to make the pipeline argument optional
def add_score(user_id, score, pipeline=None):
    if pipeline:
        pipeline.zadd('leaderboard', {user_id: score})
    else:
        client.zadd('leaderboard', {user_id: score})

def get_leaderboard(top_n=10):
    leaderboard = client.zrevrange('leaderboard', 0, top_n - 1, withscores=True)
    return [(user_id.decode('utf-8'), score) for user_id, score in leaderboard]

def get_user_rank_and_score(user_id):
    rank = client.zrevrank('leaderboard', user_id)
    score = client.zscore('leaderboard', user_id)
    return rank, score

users = [
    {'user_id': 'alice', 'data': {'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}, 'score': 50},
    {'user_id': 'bob', 'data': {'name': 'Bob', 'age': 25, 'email': 'bob@example.com'}, 'score': 80},
    {'user_id': 'charlie', 'data': {'name': 'Charlie', 'age': 35, 'email': 'charlie@example.com'}, 'score': 70}
]

# Use pipeline for batch processing of initial users
with client.pipeline() as pipeline:
    for user in users:
        add_user(user['user_id'], user['data'], pipeline=pipeline)
        add_score(user['user_id'], user['score'], pipeline=pipeline)
    pipeline.execute()

# Add a new user 'david' without using a pipeline
new_user = {'user_id': 'david', 'data': {'name': 'David', 'age': 40, 'email': 'david@example.com'}, 'score': 90}
add_user(new_user['user_id'], new_user['data'])
add_score(new_user['user_id'], new_user['score'])

# Get leaderboard
print("Leaderboard:", get_leaderboard())

# Get rank and score for each user
for user in users + [new_user]:
    rank, score = get_user_rank_and_score(user['user_id'])
    print(f"User {user['user_id']} rank: {rank}, score: {score}")
```

### Explanation of Changes:

1. **Optional Pipeline Argument**: Both `add_user` and `add_score` functions now have an optional `pipeline` argument.
2. **Conditional Logic**: When `pipeline` is provided, the function uses it for setting data. Otherwise, it falls back to `client`.
3. **New User Addition Without Pipeline**: After initializing users with a pipeline, we add a new user (`david`) without using a pipeline, ensuring both functionalities work.

This setup provides flexibility by allowing either batch processing with a pipeline or individual operations without it.